In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from scipy import stats

In [ ]:
# IMPORT THE DATASETS
df_2019 = pd.read_csv('SO2_2019_mod.csv', sep=',')
df_2020 = pd.read_csv('SO2_2020_mod.csv', sep=',')
df_2021 = pd.read_csv('SO2_2021_mod.csv', sep=',')
df_2022 = pd.read_csv('SO2_2022_mod.csv', sep=',')
df_2023 = pd.read_csv('SO2_2023_mod.csv', sep=',')

# CONCATENATE THE DATESETS

df = pd.concat([df_2019, df_2020, df_2021, df_2022, df_2023], ignore_index = True)
df

In [ ]:
#RENAME THE COLUMNS AND PRINT df
df = df.rename(columns={"Data/Ora": "DateTime", " Biossido di Zolfo - µg/m³": "Concentration"})
df


In [ ]:
# Switch the datetime into the usual format
df['DateTime'] = pd.to_datetime(df['DateTime'],format="%Y-%m-%d %H:%M:%S")
df


In [ ]:
# MISSING OR INVALID VALUES ARE DENOTED AS -999.0, WE ADDRESS THEM AS NaN
df.replace(-999.0, np.nan, inplace=True)
df

#COUNT OF NaN VALUES
print(' ')
print('There are', np.isnan(df['Concentration']).sum(), 'missing or invalid values out of', len(df))
print('The percentage of missing or invalid values is', round(np.isnan(df['Concentration']).sum()/len(df)*100,2),'%')
# SAVE THE DATASET
# df.to_csv('SO2_2019-2023_before_imputation_mod.csv', index=False)

In [ ]:
# PLOT THE DATASET
# visualize data
df['DateTime'] = pd.to_datetime(df['DateTime'])
plt.figure(figsize = (15,3))
plt.plot(df['DateTime'], df['Concentration'], linewidth = 0.3)
plt.xlabel('Year', fontsize=12)
plt.ylabel(r'$SO_{2}$ Concentration ($\mu g/m^3$)', fontsize=12)
# plt.title(r' concentration')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Save the figure in .eps and .png format
plt.savefig('so2_concentration_plot.png', dpi=300, bbox_inches='tight')
plt.savefig('so2_concentration_plot.eps', dpi=300, bbox_inches='tight', format='eps')

# Show the figure
plt.show()

In [ ]:
# Plot of the time series for every year 2019-2023
fig, axs = plt.subplots(5, 1, figsize=(12, 20))

years = [2019, 2020, 2021, 2022, 2023]

for i, year in enumerate(years):
    year_data = df[df['DateTime'].dt.year == year]
    axs[i].plot(year_data['DateTime'], year_data['Concentration'], linewidth=0.5)
    axs[i].set_xlabel('Date')
    axs[i].set_ylabel(r'Concentration ($\mu g/m^3$)')
    axs[i].set_title(f'$SO_2$ concentration for {year}')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize the values of SO2 greater than 40:
for i in range(0,len(df)):
    if (df['Concentration'].iloc[i]>40) == True:
        print(df['Concentration'].iloc[i])
        

In [ ]:
# Replace values over 40 with NaN
df['Concentration'] = np.where(df['Concentration'] > 40, np.nan, df['Concentration'])
df    

In [ ]:
# Add variable Diff
# diff x_t = y_t - y_{t-1}
df['Diff']=df['Concentration'].diff()
df

In [ ]:
# Add SgnDiff, SgnDiff1, SgnDiff2, SignDiff3
df['SgnDiff'] = 0
df['SgnDiff1'] = 0
df['SgnDiff2'] = 0
df['SgnDiff3'] = 0
df

In [ ]:
# SgnDiff3[i]=SgnDiff[i-3]
# Start da i=3, altrimenti non ho indice
#
# Idea: se Diff[i] è NaN allora SgnDiff[i] è NaN
#       se Diff[i] è positivo allora SgnDiff[i] = 1
#       altrimenti SgnDiff[i] = -1
#       idem per 
#       SgnDiff1, SgnDiff2, SgnDiff3 tenendo conto che 
#       SgnDiffj[i]=SgnDiff[i-j], j=1,2,3
#
# SgnDiff, SgnDiff1, SgnDiff2, SignDiff3
for i in range(3,len(df),1):
    # SgnDiff
    if (np.isnan(df['Diff'].iloc[i])) == True: 
        df['SgnDiff'].iloc[i] = np.nan
    if (df['Diff'].iloc[i] >= 0) == True:    
        df['SgnDiff'].iloc[i] = 1
    if (df['Diff'].iloc[i] < 0) == True:    
        df['SgnDiff'].iloc[i] = -1
    # SgnDiff1
    if (np.isnan(df['Diff'].iloc[i-1])) == True: 
        df['SgnDiff1'].iloc[i] = np.nan
    if (df['Diff'].iloc[i-1] >= 0) == True:    
        df['SgnDiff1'].iloc[i] = 1
    if (df['Diff'].iloc[i-1] < 0) == True:    
        df['SgnDiff1'].iloc[i] = -1   
    # SgnDiff2
    if (np.isnan(df['Diff'].iloc[i-2])) == True: 
        df['SgnDiff2'].iloc[i] = np.nan
    if (df['Diff'].iloc[i-2] >= 0) == True:    
        df['SgnDiff2'].iloc[i] = 1
    if (df['Diff'].iloc[i-2] < 0) == True:    
        df['SgnDiff2'].iloc[i] = -1 
    # SgnDiff3
    if (np.isnan(df['Diff'].iloc[i-3])) == True: 
        df['SgnDiff3'].iloc[i] = np.nan
    if (df['Diff'].iloc[i-3] >= 0) == True:    
        df['SgnDiff3'].iloc[i] = 1
    if (df['Diff'].iloc[i-3] < 0) == True:    
        df['SgnDiff3'].iloc[i] = -1
    # Increase Index
    i=i+1   

In [ ]:
# sistemo primi 3 valori
df['SgnDiff'].iloc[0:3] = np.nan
df['SgnDiff1'].iloc[0:3] = np.nan
df['SgnDiff2'].iloc[0:3] = np.nan
df['SgnDiff3'].iloc[0:3] = np.nan
df

In [ ]:
# Add Year, Month, DayWeek, Hour variables
df['Year'] = [d.year for d in df.DateTime]
df['Month'] = [d.strftime('%b') for d in df.DateTime]
df['DayWeek'] = [d.strftime('%a') for d in df.DateTime]
df['Hour'] = [d.hour for d in df.DateTime]
df

In [ ]:
# Creo la variable dummy Outlier che prende valore 1 se il valore di SO2 eccede 4 volte la deviazione standard
df['Outlier']=0;
std4 = 4*df['Concentration'].std();

for i in range(len(df)):
    if (df['Concentration'].iloc[i] > std4) == True:
        df['Outlier'].iloc[i] = 1
    if (np.isnan(df['Concentration'].iloc[i])) == True:
        df['Outlier'].iloc[i] = 1
    i=i+1
    
df['Outlier'].sum()

In [ ]:
# Estraggo sottodataframe dei valori Nan di SulfurDioxideRawNoOut
df_nan=df[pd.isna(df.Concentration)==True]
df_nan

In [ ]:
# Creo dataframe dei NaN divisi per Hour e Year
df_nan_year_hour=pd.DataFrame(columns=['Hour','Year','Count'])
df_nan_year_hour

In [ ]:
# Ciclo for per creare dataframe dei nan divisi per Hour e year
for hour in range(0,24,1):
    for year in range(2019,2024,1):
        count=len(df_nan[(df_nan.Year==year) & (df_nan.Hour == hour)])
        new_raw=pd.DataFrame([[hour,year,count]],columns=['Hour','Year','Count'])
        df_nan_year_hour=pd.concat([df_nan_year_hour,new_raw],ignore_index=True)
        
df_nan_year_hour

In [ ]:
#Check di aver contato bene 
df_nan_year_hour.Count.sum()

In [ ]:
# Plot NaN by Hour and Year
by='Year'
plt.figure(figsize = (6,3))
pl=plt.bar(df_nan_year_hour[df_nan_year_hour[by]==2019].Hour, \
        df_nan_year_hour[df_nan_year_hour[by]==2019].Count,  align='center')
for bar in pl:
    plt.annotate(bar.get_height(),xy=(bar.get_x(), bar.get_height()+0.8),size=11)
plt.xlabel('Hour',fontsize=14)
plt.ylabel('Count',fontsize=14)
plt.ylim(0,30)
plt.title('2019',fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Save the figure in .eps and .png format
plt.savefig('missing_data_2019.png', dpi=300, bbox_inches='tight')
plt.savefig('missing_data_2019.eps', dpi=300, bbox_inches='tight', format='eps')

plt.show()

plt.figure(figsize = (6,3))
pl=plt.bar(df_nan_year_hour[df_nan_year_hour[by]==2020].Hour, \
        df_nan_year_hour[df_nan_year_hour[by]==2020].Count,  align='center')
for bar in pl:
     plt.annotate(bar.get_height(),xy=(bar.get_x(), bar.get_height()+0.8),size=11)
plt.xlabel('Hour',fontsize=14)
plt.ylabel('Count',fontsize=14)
plt.ylim(0,30)
plt.title('2020',fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig('missing_data_2020.png', dpi=300, bbox_inches='tight')
plt.savefig('missing_data_2020.eps', dpi=300, bbox_inches='tight', format='eps')
plt.show()

plt.figure(figsize = (6,3))
pl=plt.bar(df_nan_year_hour[df_nan_year_hour[by]==2021].Hour, \
        df_nan_year_hour[df_nan_year_hour[by]==2021].Count,  align='center')
for bar in pl:
    plt.annotate(bar.get_height(),xy=(bar.get_x(), bar.get_height()+0.8),size=11)
plt.xlabel('Hour',fontsize=14)
plt.ylabel('Count',fontsize=14)
plt.ylim(0,30)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.title('2021',fontsize=14)
plt.savefig('missing_data_2021.png', dpi=300, bbox_inches='tight')
plt.savefig('missing_data_2021.eps', dpi=300, bbox_inches='tight', format='eps')
plt.show()

plt.figure(figsize = (6,3))
pl=plt.bar(df_nan_year_hour[df_nan_year_hour[by]==2022].Hour, \
        df_nan_year_hour[df_nan_year_hour[by]==2022].Count,  align='center')
for bar in pl:
    plt.annotate(bar.get_height(),xy=(bar.get_x(), bar.get_height()+0.8),size=11)
plt.xlabel('Hour',fontsize=14)
plt.ylabel('Count',fontsize=14)
plt.ylim(0,30)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.title('2022',fontsize=14)
plt.savefig('missing_data_2022.png', dpi=300, bbox_inches='tight')
plt.savefig('missing_data_2022.eps', dpi=300, bbox_inches='tight', format='eps')
plt.show()

plt.figure(figsize = (6,3))
pl=plt.bar(df_nan_year_hour[df_nan_year_hour[by]==2023].Hour, \
        df_nan_year_hour[df_nan_year_hour[by]==2023].Count,  align='center')
for bar in pl:
    plt.annotate(bar.get_height(),xy=(bar.get_x(), bar.get_height()+0.8),size=11)
plt.xlabel('Hour',fontsize=14)
plt.ylabel('Count',fontsize=14)
plt.ylim(0,30)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.title('2023',fontsize=14)
plt.savefig('missing_data_2023.png', dpi=300, bbox_inches='tight')
plt.savefig('missing_data_2023.eps', dpi=300, bbox_inches='tight', format='eps')
plt.show()


In [ ]:
# Filter out rows where 'Outlier' is equal to 1
df_noOut = df[df['Outlier'] == 0].copy()

# Reset the index of the new DataFrame
df_noOut.reset_index(drop=True, inplace=True)

df_noOut

## Boxplot of the distributions

#### Boxplot of absolute values of concentration

In [ ]:
# Variables
namex='Year'
namey='Concentration'
by='Year'
whis=1.5
outliers=True

# Boxplot
plt.figure(figsize = (8,4))
sns.boxplot(x=namex, y=namey, data=df, whis=whis, showfliers=outliers)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel(namex, fontsize=12)  # Customize the x-axis label
plt.ylabel(namey, fontsize=12)
# Save the figure in .eps and .png format
plt.savefig('boxplot_outliers.png', dpi=300, bbox_inches='tight')
plt.savefig('boxplot_outliers.eps', dpi=300, bbox_inches='tight', format='eps')


plt.show()

##### Da qui decidiamo di togliere i tre valori sopra il livello di  S02 40

In [ ]:
#Creo dataframe senza i NaN
df_no_nan = df.copy()
df_no_nan = df_no_nan[np.isnan(df['Concentration'])==False]
df_no_nan

In [ ]:
namex='Year'
namey='Concentration'
by='Year'
whis=1.5
outliers=False

# Boxplot
plt.figure(figsize = (8,4))
sns.boxplot(x=namex, y=namey, data=df, whis=whis, showfliers=outliers)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel(namex, fontsize=12)  # Customize the x-axis label
plt.ylabel(namey, fontsize=12)
# Save the figure in .eps and .png format
plt.savefig('boxplot_no_outliers.png', dpi=300, bbox_inches='tight')
plt.savefig('boxplot_no_outliers.eps', dpi=300, bbox_inches='tight', format='eps')


plt.show()


### Boxplot con db completo 

In [ ]:
# Variables
namex='Hour'
namey='Concentration'
by='Year'
whis=1.5
outliers=False

# Boxplot
fig, axes = plt.subplots(5, 1, figsize=(17,17), sharex=True, sharey=True)

sns.boxplot(x=namex, y=namey, data=df[df[by]==2019], ax=axes[0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]==2020], ax=axes[1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]==2021], ax=axes[2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]==2022], ax=axes[3], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]==2023], ax=axes[4], whis=whis, showfliers=outliers)
# Set Title
axes[0].set_title('2019', fontsize=12)
axes[1].set_title('2020', fontsize=12)
axes[2].set_title('2021', fontsize=12)
axes[3].set_title('2022', fontsize=12)
axes[4].set_title('2023', fontsize=12)

plt.show()

In [ ]:
# Creo nuovo dataset senza i 3 punti isolati --> outlier
# Imputation su 2019-2023 e gli ultimi due
# Le autocorrelazioni le facciamo sia con i salti che senza salti
# Gli ultimi due anni sono diversi da quelli precedenti

### Diff hour by year - Dataset where the record with SO2 concentration < 4std

In [ ]:
# Differenza con quello prima è che qui uso il dataset dove ho eliminato tutte le righe con SO2 
# superiore a 4 volte la deviazione standard

# Variables
namex='Hour'
namey='Diff'
by='Year'
whis=1.5
outliers=False

# Boxplot
fig, axes = plt.subplots(5, 1, figsize=(10,17), sharex=True, sharey=True)

sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]==2019], ax=axes[0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]==2020], ax=axes[1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]==2021], ax=axes[2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]==2022], ax=axes[3], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]==2023], ax=axes[4], whis=whis, showfliers=outliers)
# Set Title
axes[0].set_title('2019', fontsize=12)
axes[1].set_title('2020', fontsize=12)
axes[2].set_title('2021', fontsize=12)
axes[3].set_title('2022', fontsize=12)
axes[4].set_title('2023', fontsize=12)

plt.show()

### Box plot month por hour - db completo

In [ ]:
# Variables
namex='Hour'
namey='Diff'
by='Month'
whis=1.5
outliers=False

# Boxplot
fig, axes = plt.subplots(4,3, figsize=(17,22), sharex=True, sharey=True)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Jan'], ax=axes[0,0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Feb'], ax=axes[0,1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Mar'], ax=axes[0,2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Apr'], ax=axes[1,0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='May'], ax=axes[1,1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Jun'], ax=axes[1,2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Jul'], ax=axes[2,0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Aug'], ax=axes[2,1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Sep'], ax=axes[2,2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Oct'], ax=axes[3,0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Nov'], ax=axes[3,1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df[df[by]=='Dec'], ax=axes[3,2], whis=whis, showfliers=outliers)

for ax in axes.flat:
    ax.set_ylim(-2,3)

    
# Set Title
axes[0,0].set_title('Jan', fontsize=12)
axes[0,1].set_title('Feb', fontsize=12)
axes[0,2].set_title('Mar', fontsize=12)
axes[1,0].set_title('Apr', fontsize=12)
axes[1,1].set_title('May', fontsize=12)
axes[1,2].set_title('Jun', fontsize=12)
axes[2,0].set_title('Jul', fontsize=12)
axes[2,1].set_title('Aug', fontsize=12)
axes[2,2].set_title('Sep', fontsize=12)
axes[3,0].set_title('Oct', fontsize=12)
axes[3,1].set_title('Nov', fontsize=12)
axes[3,2].set_title('Dec', fontsize=12)

plt.show()

### Box plot - db filtrato (eliminati i valori > 4std)

In [ ]:
# Variables
namex='Hour'
namey='Diff'
by='Month'
whis=1.5
outliers=False

# Boxplot
fig, axes = plt.subplots(4,3, figsize=(17,22), sharex=True, sharey=True)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Jan'], ax=axes[0,0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Feb'], ax=axes[0,1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Mar'], ax=axes[0,2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Apr'], ax=axes[1,0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='May'], ax=axes[1,1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Jun'], ax=axes[1,2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Jul'], ax=axes[2,0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Aug'], ax=axes[2,1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Sep'], ax=axes[2,2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Oct'], ax=axes[3,0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Nov'], ax=axes[3,1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=df_noOut[df_noOut[by]=='Dec'], ax=axes[3,2], whis=whis, showfliers=outliers)

for ax in axes.flat:
    ax.set_ylim(-2,3)

    
# Set Title
axes[0,0].set_title('Jan', fontsize=12)
axes[0,1].set_title('Feb', fontsize=12)
axes[0,2].set_title('Mar', fontsize=12)
axes[1,0].set_title('Apr', fontsize=12)
axes[1,1].set_title('May', fontsize=12)
axes[1,2].set_title('Jun', fontsize=12)
axes[2,0].set_title('Jul', fontsize=12)
axes[2,1].set_title('Aug', fontsize=12)
axes[2,2].set_title('Sep', fontsize=12)
axes[3,0].set_title('Oct', fontsize=12)
axes[3,1].set_title('Nov', fontsize=12)
axes[3,2].set_title('Dec', fontsize=12)

plt.show()

### Acf e Pacf of diff by year (per prob cond successiva)

##### DB completo


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
# ACF of the increments
# REMARK: dflier è il dataset con rimossi gli outlier sopra le 4 deviazioni standard

fig, axes = plt.subplots(5,1, figsize=(17,17), sharex=True, sharey=True)

plot_acf(df[df.Year == 2019].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[0])
plot_acf(df[df.Year == 2020].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[1])
plot_acf(df[df.Year == 2021].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[2])
plot_acf(df[df.Year == 2022].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[3])
plot_acf(df[df.Year == 2023].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[4])


# Set Title
axes[0].set_title('2019', fontsize=12)
axes[1].set_title('2020', fontsize=12)
axes[2].set_title('2021', fontsize=12)
axes[3].set_title('2022', fontsize=12)
axes[4].set_title('2023', fontsize=12)


plt.show()

#### Dataset filtrato

In [ ]:
# ACF of the increments
# REMARK: df_noOut contiene tutti i valori del dataset iniziale tranne i 3 valori sopra 40
fig, axes = plt.subplots(5,1, figsize=(17,17), sharex=True, sharey=True)

plot_acf(df_noOut[df_noOut.Year == 2019].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[0])
plot_acf(df_noOut[df_noOut.Year == 2020].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[1])
plot_acf(df_noOut[df_noOut.Year == 2021].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[2])
plot_acf(df_noOut[df_noOut.Year == 2022].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[3])
plot_acf(df_noOut[df_noOut.Year == 2023].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[4])


# Set Title
axes[0].set_title('2019', fontsize=12)
axes[1].set_title('2020', fontsize=12)
axes[2].set_title('2021', fontsize=12)
axes[3].set_title('2022', fontsize=12)
axes[4].set_title('2023', fontsize=12)


plt.show()

#### Le autocorrelazioni sono molto simili, vado avanti con il db completo

### Dataset completo

In [ ]:
# PACF of the increments - db completo

fig, axes = plt.subplots(5,1, figsize=(17,17), sharex=True, sharey=True)

plot_pacf(df[df.Year == 2019].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[0])
plot_pacf(df[df.Year == 2020].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[1])
plot_pacf(df[df.Year == 2021].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[2])
plot_pacf(df[df.Year == 2022].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[3])
plot_pacf(df[df.Year == 2023].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[4])


# Set Title
axes[0].set_title('2019', fontsize=12)
axes[1].set_title('2020', fontsize=12)
axes[0].set_title('2021', fontsize=12)
axes[1].set_title('2022', fontsize=12)
axes[2].set_title('2023', fontsize=12)



plt.show()

#### Dataset completo

In [ ]:
# ACF of the increments

fig, axes = plt.subplots(4,3, figsize=(17,22), sharex=True, sharey=True)

plot_acf(df[df.Month == 'Jan'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[0,0])
plot_acf(df[df.Month == 'Feb'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[0,1])
plot_acf(df[df.Month == 'Mar'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[0,2])
plot_acf(df[df.Month == 'Apr'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[1,0])
plot_acf(df[df.Month == 'May'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[1,1])
plot_acf(df[df.Month == 'Jun'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[1,2])
plot_acf(df[df.Month == 'Jul'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[2,0])
plot_acf(df[df.Month == 'Aug'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[2,1])
plot_acf(df[df.Month == 'Sep'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[2,2])
plot_acf(df[df.Month == 'Oct'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[3,0])
plot_acf(df[df.Month == 'Nov'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[3,1])
plot_acf(df[df.Month == 'Dec'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[3,2])



# Set Title
axes[0,0].set_title('Jan', fontsize=12)
axes[0,1].set_title('Feb', fontsize=12)
axes[0,2].set_title('Mar', fontsize=12)
axes[1,0].set_title('Apr', fontsize=12)
axes[1,1].set_title('May', fontsize=12)
axes[1,2].set_title('Jun', fontsize=12)
axes[2,0].set_title('Jul', fontsize=12)
axes[2,1].set_title('Aug', fontsize=12)
axes[2,2].set_title('Sep', fontsize=12)
axes[3,0].set_title('Oct', fontsize=12)
axes[3,1].set_title('Nov', fontsize=12)
axes[3,2].set_title('Dec', fontsize=12)


#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_diff_acf_month.eps',format='eps')
#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_diff_acf_month.png',format='png')


plt.show()

#### Dataset filtrato

In [ ]:
# ACF of the increments

fig, axes = plt.subplots(4,3, figsize=(17,22), sharex=True, sharey=True)

plot_acf(df_noOut[df_noOut.Month == 'Jan'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[0,0])
plot_acf(df_noOut[df_noOut.Month == 'Feb'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[0,1])
plot_acf(df_noOut[df_noOut.Month == 'Mar'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[0,2])
plot_acf(df_noOut[df_noOut.Month == 'Apr'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[1,0])
plot_acf(df_noOut[df_noOut.Month == 'May'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[1,1])
plot_acf(df_noOut[df_noOut.Month == 'Jun'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[1,2])
plot_acf(df_noOut[df_noOut.Month == 'Jul'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[2,0])
plot_acf(df_noOut[df_noOut.Month == 'Aug'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[2,1])
plot_acf(df_noOut[df_noOut.Month == 'Sep'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[2,2])
plot_acf(df_noOut[df_noOut.Month == 'Oct'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[3,0])
plot_acf(df_noOut[df_noOut.Month == 'Nov'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[3,1])
plot_acf(df_noOut[df_noOut.Month == 'Dec'].Diff, \
        missing='conservative',lags=24, alpha=0.05, title=None, ax=axes[3,2])



# Set Title
axes[0,0].set_title('Jan', fontsize=12)
axes[0,1].set_title('Feb', fontsize=12)
axes[0,2].set_title('Mar', fontsize=12)
axes[1,0].set_title('Apr', fontsize=12)
axes[1,1].set_title('May', fontsize=12)
axes[1,2].set_title('Jun', fontsize=12)
axes[2,0].set_title('Jul', fontsize=12)
axes[2,1].set_title('Aug', fontsize=12)
axes[2,2].set_title('Sep', fontsize=12)
axes[3,0].set_title('Oct', fontsize=12)
axes[3,1].set_title('Nov', fontsize=12)
axes[3,2].set_title('Dec', fontsize=12)


#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_diff_acf_month.eps',format='eps')
#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_diff_acf_month.png',format='png')


plt.show()

### PACF

In [ ]:
# PACF of the increments -  dataset completo

fig, axes = plt.subplots(4,3, figsize=(17,22), sharex=True, sharey=True)

plot_pacf(df[df.Month == 'Jan'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[0,0])
plot_pacf(df[df.Month == 'Feb'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[0,1])
plot_pacf(df[df.Month == 'Mar'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[0,2])
plot_pacf(df[df.Month == 'Apr'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[1,0])
plot_pacf(df[df.Month == 'May'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[1,1])
plot_pacf(df[df.Month == 'Jun'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[1,2])
plot_pacf(df[df.Month == 'Jul'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[2,0])
plot_pacf(df[df.Month == 'Aug'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[2,1])
plot_pacf(df[df.Month == 'Sep'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[2,2])
plot_pacf(df[df.Month == 'Oct'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[3,0])
plot_pacf(df[df.Month == 'Nov'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[3,1])
plot_pacf(df[df.Month == 'Dec'].Diff.dropna(), \
        lags=24, alpha=0.05, title=None, ax=axes[3,2])




# Set Title
axes[0,0].set_title('Jan', fontsize=12)
axes[0,1].set_title('Feb', fontsize=12)
axes[0,2].set_title('Mar', fontsize=12)
axes[1,0].set_title('Apr', fontsize=12)
axes[1,1].set_title('May', fontsize=12)
axes[1,2].set_title('Jun', fontsize=12)
axes[2,0].set_title('Jul', fontsize=12)
axes[2,1].set_title('Aug', fontsize=12)
axes[2,2].set_title('Sep', fontsize=12)
axes[3,0].set_title('Oct', fontsize=12)
axes[3,1].set_title('Nov', fontsize=12)
axes[3,2].set_title('Dec', fontsize=12)


#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_diff_pacf_month.eps',format='eps')
#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_diff_pacf_month.png',format='png')


plt.show()

##### Remove outliers from Diff and calculate ProbabilitàCondizionata divisa per anno(tengo agosto)

##### Remove Outliers of Diff by Hour and Year

###### Definisco il flag ISOutDiff per vedere se la differenza è outlier per hour e year

In [ ]:
df[np.isnan(df.Concentration)==True]

In [ ]:
#preparo db pre imputation riprendendo i NaN ma filtrando i valori sopra 40
df_pre_imputation = df.copy()
df_pre_imputation


In [ ]:
# Outliers <Q1-1.5*IQR o >Q3+1.5*IQR

# Definisco nuovo dataframe 
df_diff=pd.DataFrame(columns=['DateTime','Concentration',\
                            'Diff','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','Year','Month','DayWeek','Hour','Outlier','IsOutDiff'])

# Calcolo quantili hour by year (senza mese agosto)
for hour in range(0,24,1):
    for year in range(2019,2024,1):
        # Estraggo sottodataframe by Hour e Year
        df_mod=df_pre_imputation[ (df_pre_imputation.Hour == hour) & \
                    (df_pre_imputation.Year == year) ] 
        # Calcolo upper e lower 1.5*IQR
        Q1=df_mod.Diff.quantile(0.25)
        Q3=df_mod.Diff.quantile(0.75)
        upper=Q3+1.5*(Q3-Q1)
        lower=Q1-1.5*(Q3-Q1)
    
        # Label se Diff è outlier (includo anche i NaN così poi li escludo) 1, else 0
        df_mod['IsOutDiff']=df_mod.apply(lambda x: 1 if ((np.isnan(x.Diff)==True) or (x.Diff > upper) or (x.Diff < lower)) else 0, axis=1)
    
        # Concateno i dataframe
        df_diff=\
        pd.concat([df_diff,df_mod],ignore_index=True)    

In [ ]:
#Check
df_diff

In [ ]:
# Probablità condizionata su dataframe senza outliers in Diff
# seleziono label ISOutDiff = 0, così non ho outliers e nemmeno Nan
SO2_probcond_df=df_diff[df_diff.IsOutDiff == 0]
SO2_probcond_df

In [ ]:
# Elimino i NaN (da Diff già passaggio prima, ma mi serve anche da SgnDiff,1,2,3)
SO2_probcond_df.dropna(subset=['Diff','SgnDiff1','SgnDiff2','SgnDiff3'],axis=0,inplace=True)
SO2_probcond_df

In [ ]:
# Check
SO2_probcond_df.isna().sum()

In [ ]:
# Boxplot Diff by Hour and Year with outliers whis=3

# Variables
namex='Hour'
namey='Diff'
by='Year'
whis=3
outliers=True

# Boxplot
fig, axes = plt.subplots(5,1, figsize=(17,17), sharex=True, sharey=True)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2019], ax=axes[0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2020], ax=axes[1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2021], ax=axes[2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2022], ax=axes[3], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2023], ax=axes[4], whis=whis, showfliers=outliers)


# Set Title
axes[0].set_title('2019', fontsize=12)
axes[1].set_title('2020', fontsize=12)
axes[2].set_title('2021', fontsize=12)
axes[3].set_title('2022', fontsize=12)
axes[4].set_title('2023', fontsize=12)

#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_box_hour_year_diff_out_3_removed.eps', format='eps')
#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_box_hour_year_diff_out_3_removed.png', format='png')

plt.show()

In [ ]:
# Boxplot Diff by Hour and Year with outliers whis=3

# Variables
namex='Hour'
namey='Diff'
by='Year'
whis=1.5
outliers=True

# Boxplot
fig, axes = plt.subplots(5,1, figsize=(17,17), sharex=True, sharey=True)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2019], ax=axes[0], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2020], ax=axes[1], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2021], ax=axes[2], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2022], ax=axes[3], whis=whis, showfliers=outliers)
sns.boxplot(x=namex, y=namey, data=SO2_probcond_df[SO2_probcond_df[by]==2023], ax=axes[4], whis=whis, showfliers=outliers)


# Set Title
axes[0].set_title('2019', fontsize=12)
axes[1].set_title('2020', fontsize=12)
axes[2].set_title('2021', fontsize=12)
axes[3].set_title('2022', fontsize=12)
axes[4].set_title('2023', fontsize=12)

#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_box_hour_year_diff_out_3_removed.eps', format='eps')
#plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_box_hour_year_diff_out_3_removed.png', format='png')

plt.show()

##### Calcolo probabilità condizionata, divisa by Year (by Hour è implicito nella probabilità condizionata con SgnDiff, SgnDiff1, SgnDiff2, SgnDiff3)

In [ ]:
# Creo dataframe delle probabilità condizionate by Year (senza agosto)
SO2_pc_df=pd.DataFrame(columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])

# ciclo for sugli anni
for year in range(2019,2024,1):
    # metto a zero le probabilità
    pos_pos_pos_pos = 0
    neg_pos_pos_pos = 0
    pos_pos_pos_neg = 0
    neg_pos_pos_neg = 0
    pos_pos_neg_pos = 0
    neg_pos_neg_pos = 0
    pos_pos_neg_neg = 0
    neg_pos_neg_neg = 0
    pos_neg_pos_pos = 0
    neg_neg_pos_pos = 0
    pos_neg_pos_neg = 0
    neg_neg_pos_neg = 0
    pos_neg_neg_pos = 0
    neg_neg_neg_pos = 0
    pos_neg_neg_neg = 0
    neg_neg_neg_neg = 0
    #
    #
    print('Year: ',year)
    # P(1 | 1, 1, 1 ) e P(-1 | 1, 1, 1 )
    pos_pos_pos_pos = round( (len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff == 1) & (SO2_probcond_df.SgnDiff1 == 1) & (SO2_probcond_df.SgnDiff2 == 1) & (SO2_probcond_df.SgnDiff3 == 1) ]) / len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff1 == 1) & (SO2_probcond_df.SgnDiff2 == 1) & (SO2_probcond_df.SgnDiff3 == 1) ]) ), 4)
    neg_pos_pos_pos = round ( 1 - pos_pos_pos_pos , 4)
    new_raw = pd.DataFrame([[year,1,1,1,1,pos_pos_pos_pos]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    new_raw = pd.DataFrame([[year,-1,1,1,1,neg_pos_pos_pos]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    #
    # P(1 | 1, 1, -1 ) e P(-1 | 1, 1, -1)
    pos_pos_pos_neg = round( (len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff == 1) & (SO2_probcond_df.SgnDiff1 == 1) & (SO2_probcond_df.SgnDiff2 == 1) & (SO2_probcond_df.SgnDiff3 == -1) ]) / len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff1 == 1) & (SO2_probcond_df.SgnDiff2 == 1) & (SO2_probcond_df.SgnDiff3 == -1) ]) ), 4)
    neg_pos_pos_neg = round ( 1 - pos_pos_pos_neg , 4)
    new_raw = pd.DataFrame([[year,1,1,1,-1,pos_pos_pos_neg]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    new_raw = pd.DataFrame([[year,-1,1,1,-1,neg_pos_pos_neg]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    #
    # P(1 | 1, -1, 1 ) e P(-1 | 1, -1, 1 )
    pos_pos_neg_pos = round( (len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff == 1) & (SO2_probcond_df.SgnDiff1 == 1) & (SO2_probcond_df.SgnDiff2 == -1) & (SO2_probcond_df.SgnDiff3 == 1) ]) / len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff1 == 1) & (SO2_probcond_df.SgnDiff2 == -1) & (SO2_probcond_df.SgnDiff3 == 1) ]) ) , 4)
    neg_pos_neg_pos = round ( 1 - pos_pos_neg_pos , 4)
    new_raw = pd.DataFrame([[year,1,1,-1,1,pos_pos_neg_pos]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    new_raw = pd.DataFrame([[year,-1,1,-1,1,neg_pos_neg_pos]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    #
    # P(1 | 1, -1, -1 ) e P(-1 | 1, -1, -1 )
    pos_pos_neg_neg = round( (len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff == 1) & (SO2_probcond_df.SgnDiff1 == 1) & (SO2_probcond_df.SgnDiff2 == -1) & (SO2_probcond_df.SgnDiff3 == -1) ]) / len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff1 == 1) & (SO2_probcond_df.SgnDiff2 == -1) & (SO2_probcond_df.SgnDiff3 == -1) ]) ) , 4)
    neg_pos_neg_neg = round ( 1 - pos_pos_neg_neg , 4)
    new_raw = pd.DataFrame([[year,1,1,-1,-1,pos_pos_neg_neg]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    new_raw = pd.DataFrame([[year,-1,1,-1,-1,neg_pos_neg_neg]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    #
    # P(1 | -1, 1, 1 ) e P(-1 | -1, 1, 1 )
    pos_neg_pos_pos = round( (len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff == 1) & (SO2_probcond_df.SgnDiff1 == -1) & (SO2_probcond_df.SgnDiff2 == 1) & (SO2_probcond_df.SgnDiff3 == 1) ]) / len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff1 == -1) & (SO2_probcond_df.SgnDiff2 == 1) & (SO2_probcond_df.SgnDiff3 == 1) ]) ) , 4)
    neg_neg_pos_pos = round ( 1 - pos_neg_pos_pos , 4)
    new_raw = pd.DataFrame([[year,1,-1,1,1,pos_neg_pos_pos]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    new_raw = pd.DataFrame([[year,-1,-1,1,1,neg_neg_pos_pos]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    #
    # P(1 | -1, 1, -1 ) e P(-1 | -1, 1, -1 )
    pos_neg_pos_neg = round( (len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff == 1) & (SO2_probcond_df.SgnDiff1 == -1) & (SO2_probcond_df.SgnDiff2 == 1) & (SO2_probcond_df.SgnDiff3 == -1) ]) / len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff1 == -1) & (SO2_probcond_df.SgnDiff2 == 1) & (SO2_probcond_df.SgnDiff3 == -1) ]) ) , 4)
    neg_neg_pos_neg = round ( 1 - pos_neg_pos_neg , 4)
    new_raw = pd.DataFrame([[year,1,-1,1,-1,pos_neg_pos_neg]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    new_raw = pd.DataFrame([[year,-1,-1,1,-1,neg_neg_pos_neg]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    #
    # P(1 | -1, -1, 1 ) e P(-1 | -1, -1, 1 )
    pos_neg_neg_pos = round( (len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff == 1) & (SO2_probcond_df.SgnDiff1 == -1) & (SO2_probcond_df.SgnDiff2 == -1) & (SO2_probcond_df.SgnDiff3 == 1) ]) / len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff1 == -1) & (SO2_probcond_df.SgnDiff2 == -1) & (SO2_probcond_df.SgnDiff3 == 1) ]) ) , 4)
    neg_neg_neg_pos = round ( 1 - pos_neg_neg_pos , 4)
    new_raw = pd.DataFrame([[year,1,-1,-1,1,pos_neg_neg_pos]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    new_raw = pd.DataFrame([[year,-1,-1,-1,1,neg_neg_neg_pos]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    #
    # P(1 | -1, -1, -1 ) e P(-1 | -1, -1, -1 )
    pos_neg_neg_neg = round( (len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff == 1) & (SO2_probcond_df.SgnDiff1 == -1) & (SO2_probcond_df.SgnDiff2 == -1) & (SO2_probcond_df.SgnDiff3 == -1) ]) / len(SO2_probcond_df[(SO2_probcond_df.Year == year) & (SO2_probcond_df.SgnDiff1 == -1) & (SO2_probcond_df.SgnDiff2 == -1) & (SO2_probcond_df.SgnDiff3 == -1) ]) ) , 4)
    neg_neg_neg_neg = round ( 1 - pos_neg_neg_neg , 4)
    new_raw = pd.DataFrame([[year,1,-1,-1,-1,pos_neg_neg_neg]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    new_raw = pd.DataFrame([[year,-1,-1,-1,-1,neg_neg_neg_neg]],columns=['Year','SgnDiff','SgnDiff1','SgnDiff2','SgnDiff3','ProbCond'])
    SO2_pc_df=pd.concat([SO2_pc_df,new_raw],ignore_index=True)
    #
    # stampo risultati
    # P(1 | 1, 1, 1 ) e P(-1 | 1, 1, 1 )
    print('pos_pos_pos_pos',pos_pos_pos_pos)
    print('neg_pos_pos_pos',neg_pos_pos_pos)
    print('\n')
    #
    # P(1 | 1, 1, -1 ) e P(-1 | 1, 1, -1)
    print('pos_pos_pos_neg',pos_pos_pos_neg)
    print('neg_pos_pos_neg',neg_pos_pos_neg)
    print('\n')
    #
    # P(1 | 1, -1, 1 ) e P(-1 | 1, -1, 1 )
    print('pos_pos_neg_pos',pos_pos_neg_pos)
    print('neg_pos_neg_pos',neg_pos_neg_pos)
    print('\n')
    #
    # P(1 | 1, -1, -1 ) e P(-1 | 1, -1, -1 )
    print('pos_pos_neg_neg',pos_pos_neg_neg)
    print('neg_pos_neg_neg',neg_pos_neg_neg)
    print('\n')
    #
    # P(1 | -1, 1, 1 ) e P(-1 | -1, 1, 1 )
    print('pos_neg_pos_pos',pos_neg_pos_pos)
    print('neg_neg_pos_pos',neg_neg_pos_pos)
    print('\n')
    #
    # P(1 | -1, 1, -1 ) e P(-1 | -1, 1, -1 )
    print('pos_neg_pos_neg',pos_neg_pos_neg)
    print('neg_neg_pos_neg',neg_neg_pos_neg)
    print('\n')
    #
    # P(1 | -1, -1, 1 ) e P(-1 | -1, -1, 1 )
    print('pos_neg_neg_pos',pos_neg_neg_pos)
    print('neg_neg_neg_pos',neg_neg_neg_pos)
    print('\n')
    #
    # P(1 | -1, -1, -1 ) e P(-1 | -1, -1, -1 )
    print('pos_neg_neg_neg',pos_neg_neg_neg)
    print('neg_neg_neg_neg',neg_neg_neg_neg) 
    print('\n')

In [ ]:
# Check 
SO2_pc_df

In [ ]:
#check
df_diff[(df_diff.IsOutDiff==1)==True]

In [ ]:
# Copy DataFrame
df_imput2=df_diff.copy(deep=True)
df_imput2

In [ ]:
# Adding variable for Imputation2
df_imput2['Concentration_Imput2']=df_imput2['Concentration']
df_imput2

In [ ]:
# Adding variable IndexLastMiss
df_imput2['IndexLastMiss'] = 0
df_imput2

In [ ]:
# per le prime 3 osservazioni non ho calcolato SgnDiff,1,2,3
# la seconda osservazione è un nan isolato
# faccio imputation con la media del precedente e successivo
# per l'imputation parto dalla  4 osservazione

In [ ]:
# Imputation 2: faccio media per primo valore missing 
df_imput2['Concentration'].iloc[1]=(df_imput2['Concentration'].iloc[0] + df_imput2['Concentration'].iloc[2])*0.5
df_imput2['Concentration_Imput2'].iloc[1]=df_imput2['Concentration'].iloc[1]
df_imput2

##### Imputation: by hour, by Year, increments without outliers, control lower and upper, control no negative values

In [ ]:
# Descriptive statistics
df_imput2.describe()

In [ ]:
# Calcolo Outliers su tutta la serie storica dei dati che ho 
# Upper threshold
Q3=df_imput2.Concentration	.quantile(0.75)
Q1=df_imput2.Concentration	.quantile(0.25)
upper=Q3+1.5*(Q3-Q1)
upper_3=Q3+3*(Q3-Q1)
upper_35=Q3+3.5*(Q3-Q1)
upper_4=Q3+4*(Q3-Q1)
print('Q3+1.5*IQR: ',upper)
print('Q3+3*IQR:' ,upper_3)
print('Q3+3.5*IQR:' ,upper_35)
print('Q3+4*IQR:' ,upper_4)
print('Max: ',df_imput2.Concentration.max())

In [ ]:
# Lower threshold
# se il dato precednte a quello da imputare è inferiore a lower
# allora prendo incremnto positivo
Q3=df_imput2.Concentration.quantile(0.75)
Q1=df_imput2.Concentration.quantile(0.25)
lower_05=Q1-0.5*(Q3-Q1)
lower_1=Q1-1*(Q3-Q1)
lower=Q1-1.5*(Q3-Q1)
print('Min:' ,df_imput2.Concentration.min())
print('Q1:' ,Q1)
print('Q1-0.5*IQR:' ,lower_05)
print('Q1-1*IQR:' ,lower_1)
print('Q1-1.5*IQR:' ,lower)

In [ ]:
# Threshold upper and lower by Year
# Creo dataframe
threshold_df=pd.DataFrame(columns=['Year','Lower','Upper'])
threshold_df

In [ ]:
# Lower and upper threshold by year
for year in range(2019,2024,1):
    Q3=df_imput2[df_imput2.Year == year].Concentration.quantile(0.75)
    Q1=df_imput2[df_imput2.Year == year].Concentration.quantile(0.25)
    upper=Q3+1.5*(Q3-Q1)
    lower=Q1
    new_raw=pd.DataFrame([[year,lower,upper]],columns=['Year','Lower','Upper'])
    threshold_df=pd.concat([threshold_df,new_raw],ignore_index=True)

# Check
threshold_df    

In [ ]:
import random

In [ ]:
#Check
df_imput2[np.isnan(df_imput2.Concentration)==True]

In [ ]:
SO2_pc_df

In [ ]:
# Seed per avere stessi risultati
random.seed(1)
# Imputation 2: increments HOUR BY YEAR

# First index of the dataset
i=df_imput2.index[4]
# Set to zero the for the last NaN of a group of consecutive NaN
k=0
# Set to zero sgndiff
sgndiff=0

# "Do While" until we end the array
while True:
    # If we have a value we increment the index
    if (np.isnan(df_imput2.Concentration.iloc[i]))==False:
        # Aggiorno dataframe
        # Diff[i]
        if (np.isnan(df_imput2.Diff.iloc[i]))==True:
            df_imput2.Diff.iloc[i] = \
            df_imput2.Concentration_Imput2.iloc[i] -\
            df_imput2.Concentration_Imput2.iloc[i-1]
        # SgnDiff[i]
        if (df_imput2.Diff.iloc[i] >= 0) == True:
            df_imput2.SgnDiff.iloc[i] = 1
        else:
            df_imput2.SgnDiff.iloc[i] = -1
        # SgnDiff1[i]
        if (df_imput2.Diff.iloc[i-1] >= 0) == True:
            df_imput2.SgnDiff1.iloc[i] = 1
        else:
            df_imput2.SgnDiff1.iloc[i] = -1
        # SgnDiff2[i]
        if (df_imput2.Diff.iloc[i-2] >= 0) == True:
            df_imput2.SgnDiff2.iloc[i] = 1
        else:
            df_imput2.SgnDiff2.iloc[i] = -1
        # SgnDiff3[i]
        if (df_imput2.Diff.iloc[i-3] >= 0) == True:
            df_imput2.SgnDiff3.iloc[i] = 1
        else:
            df_imput2.SgnDiff3.iloc[i] = -1
        # Increment the index
        i=i+1
    #    
    # If we have a NaN   
    if (np.isnan(df_imput2.Concentration.iloc[i]))==True:
        # "Do While" until we have a value in order to store the last index of a NaN of a group of consecutive NaN
        # and imputation
        while True:
            print('Index: ',i)
            # Year, Hour related to the actual NaN
            year=df_imput2.Year.iloc[i]
            hour=df_imput2.Hour.iloc[i]
            year_threshold=df_imput2.Year.iloc[i-1]
            # lower and upper threshold
            lower=threshold_df[threshold_df.Year == year_threshold].Lower.max()
            upper=threshold_df[threshold_df.Year == year_threshold].Upper.max()
            print('lower: ',lower,'upper: ',upper)
            #
#             # se è il mese di Agosto prendo SO2_pc_df per probabilità condizionata
#             if ((df_imput2.Month.iloc[i] == 'Aug') == True):
#                 print('Agosto')
#             # altrimenti (tutti gli altri mesi) prendo SO2_pc_df per probcond
#             else:      
#                 print('Altri mesi')
            while True:
                # Sample of increments, year, hour, probcond, controllo su sgndiff con lower e upper
                # sgndiff
                if ((df_imput2['Concentration_Imput2'].iloc[i-1] < lower ) == True):
                    sgndiff=1
                    print('<lower',lower)
                elif ((df_imput2['Concentration_Imput2'].iloc[i-1] > upper ) == True):  
                    sgndiff=-1
                    print('>upper',upper)
                else:
                    sgndiff = (2*stats.bernoulli.rvs( \
                    SO2_pc_df[ (SO2_pc_df.Year == year) & (SO2_pc_df.SgnDiff == 1) & \
                    (SO2_pc_df.SgnDiff1 == df_imput2.SgnDiff.iloc[i-1]) & \
                    (SO2_pc_df.SgnDiff2 == df_imput2.SgnDiff.iloc[i-2]) & \
                    (SO2_pc_df.SgnDiff3 == df_imput2.SgnDiff.iloc[i-3]) \
                    ].ProbCond ) - 1 ) 
                #    
                Increments = \
                SO2_probcond_df[ (SO2_probcond_df.Year == year) & (SO2_probcond_df.Hour == hour) & \
                (SO2_probcond_df.SgnDiff == sgndiff) ].Diff
                #
                print('sgndiff: ',sgndiff)
                print('Len Increments: ',len(Increments))
                #
                random_choice = \
                Increments[(Increments >= -df_imput2['Concentration_Imput2'].iloc[i-1] )]
                #
                print('Len random choice: ',len(random_choice))
                if ((len(random_choice)>0)==True):
                    break
            #            
            # Add random increments to have non negative values
            df_imput2['Concentration_Imput2'].iloc[i] = \
            df_imput2['Concentration_Imput2'].iloc[i-1] +\
            np.random.choice( random_choice )   
            print('Indice: ',i,\
                  'Valore precednte: ',df_imput2['Concentration_Imput2'].iloc[i-1],\
                 '\nValore attuale: ',df_imput2['Concentration_Imput2'].iloc[i])
            #
            #
            #
            # Aggiorno dataframe
            # Diff[i]
            df_imput2.Diff.iloc[i] = \
            df_imput2.Concentration_Imput2.iloc[i] -\
            df_imput2.Concentration_Imput2.loc[i-1]
            # SgnDiff[i]
            if (df_imput2.Diff.iloc[i] >= 0) == True:
                df_imput2.SgnDiff.iloc[i] = 1
            else:
                df_imput2.SgnDiff.iloc[i] = -1
            # SgnDiff1[i]
            if (df_imput2.Diff.iloc[i-1] >= 0) == True:
                df_imput2.SgnDiff1.iloc[i] = 1
            else:
                df_imput2.SgnDiff1.iloc[i] = -1
            # SgnDiff2[i]
            if (df_imput2.Diff.iloc[i-2] >= 0) == True:
                df_imput2.SgnDiff2.iloc[i] = 1
            else:
                df_imput2.SgnDiff2.iloc[i] = -1
            # SgnDiff3[i]
            if (df_imput2.Diff.iloc[i-3] >= 0) == True:
                df_imput2.SgnDiff3.iloc[i] = 1
            else:
                df_imput2.SgnDiff3.iloc[i] = -1
            #
            #
            # we are at the end of a group of consecutive NaN
            # We stop and exit from the "Do While"
            if i==df_imput2.index[len(df_imput2)-1]:      
                # Index of the last NaN in a group of consecutive NaN
                k=i                
                df_imput2['IndexLastMiss'].iloc[k] = 1
                break  
            #
            # Increase the index
            i=i+1   
            #
            # We found the first value, we stop and exit from the "Do While"
            if np.isnan((df_imput2.Concentration.iloc[i]))==False: 
                # Index of the last NaN in a group of consecutive NaN
                k=i-1                         
                df_imput2['IndexLastMiss'].iloc[k] = 1
                break 
    # We are at the end of the array, we stop and exit from the "Do While"
    if i==df_imput2.index[len(df_imput2)-1]:
        break  

Index:  27949
lower:  2.7 upper:  6.699999999999999
sgndiff:  1
Len Increments:  145
Len random choice:  145
Indice:  27949 Valore precednte:  3.6 
Valore attuale:  3.6999999999999997
Index:  27950
lower:  2.7 upper:  6.699999999999999
sgndiff:  1
Len Increments:  145
Len random choice:  145
Indice:  27950 Valore precednte:  3.6999999999999997 
Valore attuale:  3.8
Index:  27998
lower:  2.7 upper:  6.699999999999999
sgndiff:  -1
Len Increments:  173
Len random choice:  173
Indice:  27998 Valore precednte:  3.8 
Valore attuale:  3.7
Index:  28018
lower:  2.7 upper:  6.699999999999999
sgndiff:  1
Len Increments:  145
Len random choice:  145
Indice:  28018 Valore precednte:  3.4 
Valore attuale:  3.4999999999999996
Index:  28035
lower:  2.7 upper:  6.699999999999999
sgndiff:  1
Len Increments:  145
Len random choice:  145
Indice:  28035 Valore precednte:  4.2 
Valore attuale:  4.2
Index:  28215
lower:  1.5 upper:  6.5
sgndiff:  1
Len Increments:  156
Len random choice:  156
Indice:  28215

Index:  31488
lower:  2.7 upper:  6.699999999999999
sgndiff:  1
Len Increments:  147
Len random choice:  147
Indice:  31488 Valore precednte:  5.2 
Valore attuale:  5.6
Index:  31506
lower:  2.7 upper:  6.699999999999999
sgndiff:  -1
Len Increments:  179
Len random choice:  179
Indice:  31506 Valore precednte:  5.4 
Valore attuale:  5.1000000000000005
Index:  31591
lower:  2.7 upper:  6.699999999999999
sgndiff:  -1
Len Increments:  179
Len random choice:  179
Indice:  31591 Valore precednte:  2.8 
Valore attuale:  2.6999999999999997
Index:  31592
lower:  2.7 upper:  6.699999999999999
<lower 2.7
sgndiff:  1
Len Increments:  147
Len random choice:  147
Indice:  31592 Valore precednte:  2.6999999999999997 
Valore attuale:  2.9
Index:  31655
lower:  2.7 upper:  6.699999999999999
>upper 6.699999999999999
sgndiff:  -1
Len Increments:  179
Len random choice:  179
Indice:  31655 Valore precednte:  8.9 
Valore attuale:  8.8
Index:  31660
lower:  2.7 upper:  6.699999999999999
sgndiff:  1
Len Inc

Index:  37458
lower:  1.5 upper:  6.5
<lower 1.5
sgndiff:  1
Len Increments:  203
Len random choice:  203
Indice:  37458 Valore precednte:  1.2 
Valore attuale:  1.2
Index:  37611
lower:  1.5 upper:  4.5
<lower 1.5
sgndiff:  1
Len Increments:  200
Len random choice:  200
Indice:  37611 Valore precednte:  1.0 
Valore attuale:  1.0
Index:  37723
lower:  1.5 upper:  4.5
sgndiff:  1
Len Increments:  200
Len random choice:  200
Indice:  37723 Valore precednte:  2.7 
Valore attuale:  3.1
Index:  37752
lower:  1.5 upper:  4.5
sgndiff:  -1
Len Increments:  133
Len random choice:  133
Indice:  37752 Valore precednte:  2.2 
Valore attuale:  2.1000000000000005
Index:  37758
lower:  1.5 upper:  4.5
sgndiff:  1
Len Increments:  200
Len random choice:  200
Indice:  37758 Valore precednte:  2.4 
Valore attuale:  2.5999999999999996
Index:  38466
lower:  1.1 upper:  5.1000000000000005
sgndiff:  1
Len Increments:  168
Len random choice:  168
Indice:  38466 Valore precednte:  1.7 
Valore attuale:  1.9
In

Index:  42517
lower:  2.7 upper:  6.699999999999999
sgndiff:  1
Len Increments:  182
Len random choice:  182
Indice:  42517 Valore precednte:  2.8 
Valore attuale:  3.5
Index:  42921
lower:  1.5 upper:  6.5
<lower 1.5
sgndiff:  1
Len Increments:  188
Len random choice:  188
Indice:  42921 Valore precednte:  0.9 
Valore attuale:  1.1
Index:  43074
lower:  1.5 upper:  4.5
<lower 1.5
sgndiff:  1
Len Increments:  194
Len random choice:  194
Indice:  43074 Valore precednte:  1.0 
Valore attuale:  1.1
Index:  43077
lower:  1.5 upper:  4.5
sgndiff:  1
Len Increments:  194
Len random choice:  194
Indice:  43077 Valore precednte:  1.6 
Valore attuale:  1.8
Index:  43186
lower:  1.5 upper:  4.5
sgndiff:  1
Len Increments:  194
Len random choice:  194
Indice:  43186 Valore precednte:  2.5 
Valore attuale:  2.6
Index:  43215
lower:  1.5 upper:  4.5
sgndiff:  1
Len Increments:  194
Len random choice:  194
Indice:  43215 Valore precednte:  2.8 
Valore attuale:  3.0999999999999996
Index:  43221
lower

In [ ]:
# Descriptive statistics
df_imput2.describe()

In [ ]:
df_imput2[np.isnan(df_imput2.Concentration)==True]

In [ ]:
df_plot = df_diff.copy()
df_plot

In [ ]:
df_plot['Concentration_imputed']=df_imput2['Concentration_Imput2']
df_plot

In [ ]:
# Plot

df_plot = df_plot.sort_values(by='DateTime')

df_plot_2022 = df_plot[df_plot['DateTime'].dt.year==2022]
plt.figure(figsize=(20,5))
plt.plot(df_plot_2022.DateTime, df_plot_2022.Concentration_imputed, color='red')
plt.plot(df_plot_2022.DateTime, df_plot_2022.Concentration)
plt.legend(['imputed','original'])
plt.gca().set(ylabel='Concentration $(\mu g/m^3)$')

plt.savefig('plot_imputed_2022.eps', format='eps')
plt.show()

In [ ]:
# Filter data for years from 2019 to 2023
df_plot_filtered = df_plot[(df_plot['DateTime'].dt.year >= 2019) & (df_plot['DateTime'].dt.year <= 2023)]

# Sort data by DateTime
df_plot_filtered = df_plot_filtered.sort_values(by='DateTime')

# Create the plot
plt.figure(figsize = (20,5))

plt.plot(df_plot_filtered.DateTime, df_plot_filtered.Concentration_imputed, color='red', label='Imputed', linewidth = 0.3)
plt.plot(df_plot_filtered.DateTime, df_plot_filtered.Concentration, label='Original', linewidth = 0.2)

# Add legend, labels, and save
plt.legend( fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.ylabel(r'$SO_{2}$ Concentration ($\mu g/m^3$)', fontsize=14)
# plt.title(r' concentration')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig('plot_imputed_2019_2023.png', dpi=300, bbox_inches='tight')
plt.savefig('plot_imputed_2019_2023.eps', dpi=300, bbox_inches='tight', format='eps')
# plt.savefig('plot_imputed_2019_2023.eps', format='eps')
plt.show()


## Detail plot sulla base dei gruppi di missing consecutivi

In [ ]:
prova1=df_imput2[(df_imput2.Year == 2019)]

prova1 = prova1.sort_values(by='DateTime')

prova1[np.isnan(prova1.Concentration)==True]


In [ ]:
import matplotlib.dates as mdates
start = 34870
end = 34970

df_imput2 = df_imput2.sort_values(by='DateTime')

plt.figure(figsize=(20, 5))
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration_Imput2.iloc[start:end], color='red')
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration.iloc[start:end])
plt.legend(['Imputed', 'Original'], fontsize=14)

# Format the x-axis to show full date and time (YYYY-MM-DD HH:mm)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xlabel('Year', fontsize=14)
plt.ylabel(r'$SO_{2}$ Concentration ($\mu g/m^3$)', fontsize=14)

ticks, labels = plt.xticks()
plt.xticks(ticks[:-1], labels[:-1])  # Exclude the last tick and its label
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Save the figure
plt.savefig('focus_2022_imputation.png', dpi=300, bbox_inches='tight')
plt.savefig('focus_2022_imputation.eps', dpi=300, bbox_inches='tight', format='eps')

plt.show()


In [ ]:
import matplotlib.dates as mdates
start= 42797
end=42860

df_imput2 = df_imput2.sort_values(by='DateTime')

plt.figure(figsize=(20, 5))
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration_Imput2.iloc[start:end], color='red')
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration.iloc[start:end])
plt.legend(['Imputed', 'Original'], fontsize=14)

# Format the x-axis to show full date and time (YYYY-MM-DD HH:mm)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xlabel('Year', fontsize=14)
plt.ylabel(r'$SO_{2}$ Concentration ($\mu g/m^3$)', fontsize=14)

ticks, labels = plt.xticks()
# plt.xticks(ticks[:-1], labels[:-1])  # Exclude the last tick and its label
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Save the figure
plt.savefig('focus_2023_imputation.png', dpi=300, bbox_inches='tight')
plt.savefig('focus_2023_imputation.eps', dpi=300, bbox_inches='tight', format='eps')

plt.show()

In [ ]:
# Detail Plot
start= 31110
end=31200

df_imput2 = df_imput2.sort_values(by='DateTime')

plt.figure(figsize=(15,5))
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration_Imput2.iloc[start:end], color='red')
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration.iloc[start:end])
plt.legend(['imputed','original'])
plt.gca().set(ylabel='Concentration $(\mu g/m^3)$', xlabel='Date')

# plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_detail_16augsep.eps', format='eps')
# plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_detail_16augsep.png', format='png')

plt.show()

In [ ]:
#  figure per tesi
import matplotlib.dates as mdates
start= 31110
end=31200

df_imput2 = df_imput2.sort_values(by='DateTime')

plt.figure(figsize=(20, 5))
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration_Imput2.iloc[start:end], color='red')
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration.iloc[start:end])
plt.legend(['Imputed', 'Original'], fontsize=14)

# Format the x-axis to show full date and time (YYYY-MM-DD HH:mm)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xlabel('Year', fontsize=14)
plt.ylabel(r'$SO_{2}$ Concentration ($\mu g/m^3$)', fontsize=14)

ticks, labels = plt.xticks()
plt.xticks(ticks[:-1], labels[:-1])  # Exclude the last tick and its label
plt.xticks(fontsize=13)
plt.yticks(fontsize=14)

# Save the figure
plt.savefig('focus_2022_2_imputation.png', dpi=300, bbox_inches='tight')
plt.savefig('focus_2022_2_imputation.eps', dpi=300, bbox_inches='tight', format='eps')

plt.show()

In [ ]:
# Detail Plot
start= 400
end=600


plt.figure(figsize=(15,5))
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration_Imput2.iloc[start:end], color='red')
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration.iloc[start:end])
plt.legend(['imputed','original'])
plt.gca().set(ylabel='Concentration $(\mu g/m^3)$', xlabel='Date')
d
# plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_detail_16augsep.eps', format='eps')
# plt.savefig('C:\\Users\\Fujitsu\\Nextcloud\\Shared\\Degrado_tesi_Kety_Pini\\Tesi\\Figures\\Imputation\\so2_detail_16augsep.png', format='png')

plt.show()

In [ ]:
#  figure per tesi
import matplotlib.dates as mdates
start= 5240
end=5550

df_imput2 = df_imput2.sort_values(by='DateTime')

plt.figure(figsize=(20, 5))
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration_Imput2.iloc[start:end], color='red')
plt.plot(df_imput2.DateTime.iloc[start:end], df_imput2.Concentration.iloc[start:end])
plt.legend(['Imputed', 'Original'], fontsize=14)

# Format the x-axis to show full date and time (YYYY-MM-DD HH:mm)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xlabel('Year', fontsize=16)
plt.ylabel(r'$SO_{2}$ Concentration ($\mu g/m^3$)', fontsize=16)

# ticks, labels = plt.xticks()
# plt.xticks(ticks[:-1], labels[:-1])  # Exclude the last tick and its label
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Save the figure
plt.savefig('focus_2019_imputation.png', dpi=300, bbox_inches='tight')
plt.savefig('focus_2019_imputation.eps', dpi=300, bbox_inches='tight', format='eps')

plt.show()

In [ ]:
df_imput2.to_csv('SO2_2019-2023_post_imputation_by_year.csv', index=False)

### Statistical Test, mean, variance, and Kolmogorov-Smirnov

In [ ]:
# Creo dataframe per risultati dei test su media, varianza, Kolmogorov-Smirnov
# divisi per anno e ora
SO2_test_df=pd.DataFrame(columns=['Year','Hour','Var','Mean','KS'])
SO2_test_df

In [ ]:
# Ciclo for per KS test by Year and Hour
for year in range(2019,2024,1):
    for hour in range(0,24,1):
        data1=df_imput2[(df_imput2.Year == year) &\
                                                     (df_imput2.Hour == hour)]\
            .Concentration
        data2=df_imput2[(df_imput2.Year == year) &\
                                                     (df_imput2.Hour == hour)]\
            .Concentration_Imput2
        # Var test
        statistics_Var,pvalue_Var=stats.levene(data1.dropna(),data2)
        # T test
        statistics_Mean,pvalue_Mean=stats.ttest_ind(data1,data2,nan_policy='omit')
        # KS test
        statistics_KS,pvalue_KS=stats.ks_2samp(data1,data2)     
        # Add raw to dataframe
        new_raw=pd.DataFrame([[year,hour,pvalue_Var,pvalue_Mean,pvalue_KS]],columns=['Year','Hour','Var','Mean','KS'])
        SO2_test_df=pd.concat([SO2_test_df,new_raw],ignore_index=True)

# Check
SO2_test_df

In [ ]:
# Levene Test of variance
# H_0: same variance
# H_1: different variance
# Reject the null hypothesis if pvalue < 0.05
SO2_test_df[SO2_test_df.Var < 0.05]

In [ ]:
# T test for mean
# H_0: same mean
# H_1: different mean
# Reject the null hypothesis if pvalue < 0.05
SO2_test_df[SO2_test_df.Mean < 0.05]

In [ ]:
# KS test of goodness of fit
# H_0: same distribution
# H_1: different distribution
# Reject the null hypothesis if pvalue < 0.05
SO2_test_df[SO2_test_df.KS < 0.05]

In [ ]:
# Plot p-values of KS by Hour and Year
width=4.5
marker='.'

plt.figure(figsize=(10,5))
plt.axhline(y=0.01, xmin=0, xmax=24,color='blue')
plt.axhline(y=0.05, xmin=0, xmax=24,color='red')
plt.scatter(SO2_test_df[SO2_test_df.Year == 2019].Hour,SO2_test_df[SO2_test_df.Year == 2019].Mean,marker=marker,linewidths=width)
plt.scatter(SO2_test_df[SO2_test_df.Year == 2020].Hour,SO2_test_df[SO2_test_df.Year == 2020].Mean,marker=marker,linewidths=width)
plt.scatter(SO2_test_df[SO2_test_df.Year == 2021].Hour,SO2_test_df[SO2_test_df.Year == 2021].Mean,marker=marker,linewidths=width)
plt.scatter(SO2_test_df[SO2_test_df.Year == 2022].Hour,SO2_test_df[SO2_test_df.Year == 2022].Mean,marker=marker,linewidths=width)
plt.scatter(SO2_test_df[SO2_test_df.Year == 2023].Hour,SO2_test_df[SO2_test_df.Year == 2023].Mean,marker=marker,linewidths=width)


plt.legend(['0.01','0.05','2019','2020','2021','2022','2023'],\
           bbox_to_anchor=(1.0, 1.0), loc='upper left',fontsize=12)


plt.xlabel('Hour',fontsize=16)
plt.ylabel('p-value',fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.savefig('so2_ks_mean_test.png', dpi=300, bbox_inches='tight')
plt.savefig('so2_ks_mean_test.eps', dpi=300, bbox_inches='tight', format='eps')



plt.show()

In [ ]:
# Plot p-values of KS by Hour and Year
width=4.5
marker='.'

plt.figure(figsize=(10,5))
plt.axhline(y=0.01, xmin=0, xmax=24,color='blue')
plt.axhline(y=0.05, xmin=0, xmax=24,color='red')
plt.scatter(SO2_test_df[SO2_test_df.Year == 2019].Hour,SO2_test_df[SO2_test_df.Year == 2019].Var,marker=marker,linewidths=width)
plt.scatter(SO2_test_df[SO2_test_df.Year == 2020].Hour,SO2_test_df[SO2_test_df.Year == 2020].Var,marker=marker,linewidths=width)
plt.scatter(SO2_test_df[SO2_test_df.Year == 2021].Hour,SO2_test_df[SO2_test_df.Year == 2021].Var,marker=marker,linewidths=width)
plt.scatter(SO2_test_df[SO2_test_df.Year == 2022].Hour,SO2_test_df[SO2_test_df.Year == 2022].Var,marker=marker,linewidths=width)
plt.scatter(SO2_test_df[SO2_test_df.Year == 2023].Hour,SO2_test_df[SO2_test_df.Year == 2023].Var,marker=marker,linewidths=width)


plt.legend(['0.01','0.05','2019','2020','2021','2022','2023'],\
           bbox_to_anchor=(1.0, 1.0), loc='upper left',fontsize=12)


plt.xlabel('Hour',fontsize=16)
plt.ylabel('p-value',fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.savefig('so2_ks_var_test.png', dpi=300, bbox_inches='tight')
plt.savefig('so2_ks_var_test.eps', dpi=300, bbox_inches='tight', format='eps')



plt.show()